In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import json
import os
import re
import pandas as pd
from src import etl

In [12]:
#parse configuration
with open('config/data-params.json') as f:
    train = json.load(f)
sitemap = train['sitemap']
train_outpath = train['outpath']
num = train['num']
cat = train['categories']

In [13]:
apps= os.listdir(train_outpath+'/'+cat[0])

In [14]:
app_smalis = [etl.smalis_from_paths(etl.get_smali_paths(train_outpath+'/'+cat[0]+'/'+apps[i])) for i in range(len(apps))]

In [15]:
smalis = pd.DataFrame(app_smalis)

### Matrix A

In [16]:
def find_apis(block):
    return re.findall('invoke-\w+ {.*}, (.*?)\\(', block)

In [17]:
def smali2apis(row):
    smalis = '\n'.join(row.dropna())
    return set(find_apis(smalis))

In [4]:
def construct_A(apis):
    mlb = MultiLabelBinarizer()
    A = mlb.fit_transform(apis)
    return A, mlb.classes_

In [18]:
apis = smalis.apply(smali2apis, axis = 1)

In [19]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

In [56]:
A = mlb.fit_transform(apis)

In [57]:
A #obfuscated apis

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 1],
       [1, 1, 1, ..., 0, 1, 0]])

### Matrix B

In [23]:
def find_blocks(smali):
    return re.findall( '\.method([\S\s]*?)\.end method', smali)

In [24]:
def smali2blocks(row):
    smalis = '\n'.join(row.dropna())
    return list(set(find_blocks(smalis)))

In [46]:
def block2apis(block):
    apis = set(re.findall('invoke-\w+ {.*}, (.*?)\\(', block))
    for api in apis:
        if api not in B_dict.keys():
            B_dict[api] = apis
        else:
            B_dict[api] = B_dict[api].union(apis)

In [3]:
def construct_B(smalis):
    B_dict = {}
    blocks = smalis.apply(smali2blocks, axis = 1).explode().reset_index().iloc[:,1]
    blocks.dropna().apply(block2apis)
    mlb = MultiLabelBinarizer()
    return mlb.fit_transform(B_dict)

In [45]:
B_dict = {}

In [25]:
blocks = smalis.apply(smali2blocks, axis = 1).explode().reset_index().iloc[:,1]

In [48]:
blocks.dropna().apply(block2apis)

0         None
1         None
2         None
3         None
4         None
          ... 
668136    None
668137    None
668138    None
668139    None
668140    None
Name: 0, Length: 668134, dtype: object

In [51]:
mlb_b = MultiLabelBinarizer()

In [59]:
B = mlb_b.fit_transform(B_dict)

In [60]:
B

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

### Matrix P

In [67]:
def package(api):
    return re.search('(\[*[ZBSCFIJD]|\[*L[\w\/$-]+;)->', api)[1]

In [5]:
def construct_P(apis):
    api_df = pd.DataFrame({'api':apis}).dropna()
    api_df['package'] = api_df.api.apply(package)
    P_dict = api_df.groupby('package')['api'].apply(set).to_dict()
    mlb = MultiLabelBinarizer()
    return mlb.fit_transform(P_dict)

In [64]:
api_df = pd.DataFrame({'api':mlb.classes_}).dropna()

In [68]:
api_df['package'] = api_df.api.apply(package)

In [72]:
P_dict = api_df.groupby('package')['api'].apply(set).to_dict()

In [74]:
mlb_p = MultiLabelBinarizer()
P = mlb_p.fit_transform(P_dict)

In [75]:
P

array([[1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [6]:
def construct_matrices():
    smalis = pd.DataFrame(app_smalis)
    apis = smalis.apply(smali2apis, axis = 1)
    A, all_apis = construct_A(apis)
    B = construct_B(smalis)
    P = construct_P(all_apis)
    return A, B, P